# HTML変換関数の定義

In [ ]:

html_void_elements = ["area", "base", "br", "col", "embed", "hr", "img", "input", "link", "meta", "param", "source", "track", "wbr"]

def jsonmlobj_to_html(obj, use_selfclose=False, void_elements=html_void_elements):
    """
    JSONMLオブジェクトをHTML文字列に変換する

    obj: JSONMLをjson.loadsやyaml.load等で読み込んだオブジェクト
    use_selfclose: 自己終了タグに/を付けるかどうか
    void_elements: 自己終了タグ(void要素)のリスト
    """

    # 以下のいずれかに該当する場合はエラー
    # - objがリストでない
    # - objが空リスト
    # - objの最初の要素が文字列でない
    # - objの最初の要素が空文字列
    if not isinstance(obj, list) or len(obj) == 0 or not isinstance(obj[0], str) or obj[0] == "":
        raise Exception("Invalid JSONML object. The object must be a non-empty list with a string as the first element.")

    output = ""
    tag = ""
    attr = ""
    start = 1
    is_selfclose = False

    # タグ名を取得し、自己終了タグ(void要素)かどうかを判定
    tag = obj[0].lower()
    if tag in void_elements:
        is_selfclose = True
    elif tag.endswith("/"):
        tag = tag[:-1]
        is_selfclose = True
    output += f"<{tag}"

    # 属性を示す辞書型データが存在する場合は属性を取得
    if len(obj) > 1 and isinstance(obj[1], dict):
        attr = " " + (" ".join(f'{k}="{v}"' for k, v in obj[1].items()))
        start = 2

    # 開始タグを出力
    output += f"{attr} />" if is_selfclose and use_selfclose else f"{attr}>"

    # (自己終了タグでない場合は)子要素及びコンテンツを取得して出力
    if not is_selfclose:
        for i in range(start, len(obj)):
            # リストの場合は子要素なので再帰呼び出し
            if isinstance(obj[i], list):
                output += jsonmlobj_to_html(obj[i], use_selfclose, void_elements)

            # 属性以外の場所で辞書型データが存在する場合はエラー
            elif isinstance(obj[i], dict):
                raise Exception(f"Invalid JSONML object. Unexpected dictionary at position {i}. Dictionaries are only allowed as the second element to specify attributes.")

            # リストでも辞書でもない場合はコンテンツとして出力
            else:
                output += str(obj[i])

        # 終了タグを出力
        output += f"</{tag}>"

    # 自己終了タグなのに子要素やコンテンツが指定されている場合はエラー
    elif len(obj) > start:
        raise Exception("Invalid JSONML object. Self-closing tags should not have children.")

    return output

# テスト用データの定義と変換

In [ ]:
import yaml

jsonml_yaml = """
- html
- {lang: ja}
- - head
  - [meta, {charset: utf-8}]
  - [title, JSON Markup Language]
- - body
  - [h1, JSON Markup Language]
  - - div
    - - p
      - >-
        JSON Markup Language (JSONML) is a lightweight markup language
        used to represent HTML or XML documents in JSON format.
      - - font
        - {color: red}
        - Hello, World!
      - [font, {color: blue}, "Goodbye, World!"]
      - Goodbye, World!
"""

jsonml_obj = yaml.load(jsonml_yaml, Loader=yaml.FullLoader)

htmlstr = jsonmlobj_to_html(jsonml_obj)


# HTMLを出力するテスト

In [ ]:
from lxml import etree

htmlobj = etree.HTML(htmlstr)  # type: ignore
prettyhtml = etree.tostring(htmlobj, encoding="unicode", pretty_print=True).replace("/>", ">")

print(prettyhtml)


# ブラウザに表示するテスト

In [ ]:
import IPython.display as display

display.HTML(htmlstr)  # type: ignore